# Prevenzione della Rottura di Stock con Service Level nella Supply Chain

In questo notebook presentiamo una strategia per prevenire la rottura di stock (stock-out) basata su:
- la previsione del consumo giornaliero
- la conoscenza del lead time del fornitore
- l’introduzione di un livello di servizio (service level) per gestire l’incertezza

Utilizzeremo dati simulati per mostrare come funziona il nostro algoritmo predittivo.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import norm

## Parametri

Impostiamo:
- stock iniziale
- previsioni giornaliere
- lead time
- livello di servizio desiderato


In [ ]:
np.random.seed(42)

giorni = 60
stock_iniziale = 200
lead_time = 2

service_level = 0.95
z = norm.ppf(service_level)

vendite_giornaliere = np.random.poisson(5, size=giorni)
date = pd.date_range(start="2025-01-01", periods=giorni)
df = pd.DataFrame({"data": date, "vendita_prevista": vendite_giornaliere})

## Simulazione del consumo

Simuliamo l’evoluzione dello stock nel tempo a partire dallo stock iniziale.

In [ ]:
df["stock"] = stock_iniziale - df["vendita_prevista"].cumsum()

## Scorta di Sicurezza (Safety Stock)

Calcoliamo la scorta di sicurezza come:
\[
\text{safety\_stock} = z \cdot \sigma_{\text{domanda}}
\]
dove:
- \( z \) è il quantile della normale standard (es. 1.645 per 95%)
- \( \sigma \) è la deviazione standard della domanda durante il lead time

In [ ]:
domanda_lead_time = vendite_giornaliere[:lead_time]
std_domanda = np.std(domanda_lead_time)
media_domanda = int(np.mean(domanda_lead_time))
safety_stock = int(z * std_domanda)

livello_riordino = media_domanda + safety_stock

## Identificazione del giorno di ordine

Troviamo il primo giorno in cui lo stock previsto scende sotto il livello di riordino.

In [ ]:
df["trigger_ordine"] = df["stock"] <= livello_riordino
idx_trigger = df[df["trigger_ordine"]].index.min()
data_ordine = df.loc[idx_trigger, "data"] if idx_trigger is not None else None

## Simulazione "What-if" con ordine effettuato

Supponiamo di effettuare l’ordine nella data suggerita e che arrivi dopo `lead_time` giorni con una quantità fissa.

In [ ]:
qta_ordine = stock_iniziale
df["stock_whatif"] = stock_iniziale
ordine_evaso = False

for i in range(1, len(df)):
    df.loc[i, "stock_whatif"] = (
        df.loc[i - 1, "stock_whatif"] - df.loc[i, "vendita_prevista"]
    )

    data_corrente = df.loc[i, "data"]
    data_arrivo = data_ordine + pd.Timedelta(days=lead_time) if data_ordine else None

    if not ordine_evaso and data_arrivo and data_corrente >= data_arrivo:
        df.loc[i, "stock_whatif"] += qta_ordine
        ordine_evaso = True

## Visualizzazione

Grafichiamo:
- l’evoluzione dello stock senza intervento
- il livello di riordino
- la data d’ordine
- l’effetto dell’ordine (scenario what-if)

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df["data"], df["stock"], label="Stock senza ordine")
plt.plot(df["data"], df["stock_whatif"], label="Stock con ordine", linestyle="--")
plt.axhline(
    livello_riordino, color="orange", linestyle="--", label="Livello di Riordino"
)

if data_ordine:
    plt.axvline(data_ordine, color="green", linestyle="--", label="Data Ordine")
    plt.axvline(
        data_ordine + pd.Timedelta(days=lead_time),
        color="blue",
        linestyle=":",
        label="Arrivo Ordine",
    )

plt.xlabel("Data")
plt.ylabel("Stock")
plt.title("Andamento dello Stock con Strategia di Riordino e Livello di Servizio")
plt.legend()
# plt.grid(True)
plt.tight_layout()
plt.show()

## Conclusioni

L’introduzione del livello di servizio consente di tenere conto dell’incertezza nella domanda.

➡️ La scorta di sicurezza riduce il rischio di rottura di stock  
➡️ La data d’ordine viene anticipata per garantire una disponibilità sufficiente

Questo tipo di logica può essere esteso facilmente a:
- prodotti multipli
- variabilità nel lead time
- simulazioni Monte Carlo per validazione